In [1]:
# basic imports
import tensorflow as tf 
import torch
import io
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from keras.utils import dataset_utils
import matplotlib.pyplot as plt

import baseline_config

c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def create_dataset(subset):
    image_paths, labels, class_names = dataset_utils.index_directory(
            baseline_config.dataset_path + subset,
            labels="inferred",
            formats=('.pt'),
            class_names=None,
            shuffle=False,
            seed=42,
            follow_links=True)

    def paths_and_labels_to_dataset(image_paths,labels,num_classes):
        path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
        img_ds = path_ds.map(
            lambda path: tf.io.read_file(path) , num_parallel_calls=tf.data.AUTOTUNE
        )
        label_ds = dataset_utils.labels_to_dataset(labels, True, num_classes)
        img_ds = tf.data.Dataset.zip((img_ds, label_ds))
        return img_ds

    dataset = paths_and_labels_to_dataset(
        image_paths=image_paths,
        labels=labels,
        num_classes=len(class_names))
    
    return dataset, class_names

train_dataset, class_names = create_dataset('TRAIN/')
test_dataset,_             = create_dataset('TEST/')
validation_dataset,_       = create_dataset('VALIDATION/')

Found 12384 files belonging to 5 classes.
Found 487 files belonging to 5 classes.
Found 384 files belonging to 5 classes.


In [23]:
def dataset_tranforms(image,label):
    image = image.detach().numpy()
    image = torch.load(io.BytesIO(image.numpy()))
    return image,label

train_dataset = train_dataset.map(dataset_tranforms) 
test_dataset  = test_dataset.map(dataset_tranforms) 
validation_dataset = validation_dataset.map(dataset_tranforms) 

for melspec, label in train_dataset.take(3):
    # melspec = torch.load(io.BytesIO(melspec.numpy())).numpy()
    print(melspec, label)

AttributeError: in user code:

    File "C:\Users\Andrew\AppData\Local\Temp\ipykernel_45216\2541079169.py", line 2, in dataset_tranforms  *
        image = image.detach().numpy()

    AttributeError: 'Tensor' object has no attribute 'detach'


In [7]:
model = keras.Sequential(
    [
        keras.Input(shape=(313,128,1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(len(class_names), activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 311, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 155, 63, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 153, 61, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 76, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 145920)            0         
                                                                 
 dropout (Dropout)           (None, 145920)            0

In [19]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(train_dataset, validation_data=validation_dataset, batch_size=batch_size, epochs=epochs)

Epoch 1/15


ValueError: in user code:

    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\input_spec.py", line 250, in assert_input_